In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation
# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [20]:
# 데이터를 읽어온다.
df1 = pd.read_csv('./data/sonar.csv', header=None)
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [21]:
# 문자열 데이터를 숫자로 변환한다.
encoder1 = LabelEncoder()
df1[60] = encoder1.fit_transform(df1[60])
df1

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,0
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,0
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,0
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,0


In [22]:
# 입력과 결과로 나눈다.
X = df1.drop(60, axis=1)
y = df1[60]

In [23]:
# 결과 데이터의 종류 수를 파악한다.
len(y.value_counts())

2

In [24]:
# 표준화
scaler1 = StandardScaler()
X = scaler1.fit_transform(X)
display(X)

array([[-0.39955135, -0.04064823, -0.02692565, ...,  0.06987027,
         0.17167808, -0.65894689],
       [ 0.70353822,  0.42163039,  1.05561832, ..., -0.47240644,
        -0.44455424, -0.41985233],
       [-0.12922901,  0.60106749,  1.72340448, ...,  1.30935987,
         0.25276128,  0.25758223],
       ...,
       [ 1.00438083,  0.16007801, -0.67384349, ...,  0.90652575,
        -0.03913824, -0.67887143],
       [ 0.04953255, -0.09539176,  0.13480381, ..., -0.00759783,
        -0.70402047, -0.34015415],
       [-0.13794908, -0.06497869, -0.78861924, ..., -0.6738235 ,
        -0.29860448,  0.99479044]])

In [25]:
# 2진 분류 옵션들
out_nodes = 1
loss_function = 'binary_crossentropy'
activation_function = 'sigmoid'

# 다중 분류 옵션들
# out_nodes = len(y.value_counts())
# loss_function = 'categorical_crossentropy'
# activation_function = 'softmax'

# 회귀
# 출력 결과를 하나만 뽑아서 Series로 나왔다면...
# 1로 설정한다.
# if type(y) is pd.core.series.Series:
#     out_nodes = 1
# else : 
#     out_nodes = y.shape[1]
    
# loss_function = 'mean_squared_error'

In [26]:
# 입력노드의 개수
# 입력 데이터 행 하나의 컬럼의 개수
input_size = X.shape[1]
input_size

60

### 비율 설정
- 주어진 데이터를 랜덤하게 섞은 후 주어진 비율 만큼 학습셋과 검증셋을 나눠 학습과 평가를 실시한다.
- 학습시 표시하는 손실률과 정확도에서 검증셋에 대한 평가 결과도 확인할 수 있다.

In [28]:
# 신경망 설계
model = Sequential()

model.add(Dense(24, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation(activation_function))

In [29]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                1464      
                                                                 
 activation_3 (Activation)   (None, 24)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                250       
                                                                 
 activation_4 (Activation)   (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                                 
Total params: 1,725
Trainable params: 1,725
Non-traina

In [30]:
# 학습
# validation_split : 주어진 데이터에서 사용할 검증데이터 비율
model.fit(X, y, epochs=200, batch_size=10, validation_split=0.3)

Epoch 1/200
15/15 [==============================] - 1s 15ms/step - loss: 0.7105 - accuracy: 0.6138 - val_loss: 0.7941 - val_accuracy: 0.2222
Epoch 2/200
15/15 [==============================] - 0s 7ms/step - loss: 0.5971 - accuracy: 0.7517 - val_loss: 0.9223 - val_accuracy: 0.1746
Epoch 3/200
15/15 [==============================] - 0s 6ms/step - loss: 0.5195 - accuracy: 0.8483 - val_loss: 1.0362 - val_accuracy: 0.1587
Epoch 4/200
15/15 [==============================] - 0s 6ms/step - loss: 0.4581 - accuracy: 0.8621 - val_loss: 1.1604 - val_accuracy: 0.1587
Epoch 5/200
15/15 [==============================] - 0s 7ms/step - loss: 0.4069 - accuracy: 0.8897 - val_loss: 1.2988 - val_accuracy: 0.1587
Epoch 6/200
15/15 [==============================] - 0s 6ms/step - loss: 0.3652 - accuracy: 0.9103 - val_loss: 1.4452 - val_accuracy: 0.1587
Epoch 7/200
15/15 [==============================] - 0s 6ms/step - loss: 0.3314 - accuracy: 0.9241 - val_loss: 1.5712 - val_accuracy: 0.1429
Epoch 8/200


- loss, accuracy : 학습 데이터를 통한 평가
- val_loss, val_accuracy : 검증 데이터를 통한 경가
- 학습 데이터의 평가 결과는 갈수록 좋아지는데 검증 데이터의 평가 결과가 갈수록 나빠진다면 과적합이 된 것으로 봐야 한다.

### 검증용 데이터가 있을 경우
- 본 예제에서는 전체 데이터를 학습과 검증용으로 나눠서 작성한다.

In [33]:
# 학습용과 검증용으로 나눈다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [34]:
# 신경망 설계
model = Sequential()

model.add(Dense(24, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation(activation_function))

In [35]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                1464      
                                                                 
 activation_6 (Activation)   (None, 24)                0         
                                                                 
 dense_7 (Dense)             (None, 10)                250       
                                                                 
 activation_7 (Activation)   (None, 10)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
 activation_8 (Activation)   (None, 1)                 0         
                                                                 
Total params: 1,725
Trainable params: 1,725
Non-traina

In [36]:
# 학습
model.fit(X_train, y_train, epochs=200, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/200
15/15 [==============================] - 1s 14ms/step - loss: 0.6013 - accuracy: 0.7034 - val_loss: 0.6205 - val_accuracy: 0.7302
Epoch 2/200
15/15 [==============================] - 0s 7ms/step - loss: 0.5384 - accuracy: 0.7793 - val_loss: 0.5930 - val_accuracy: 0.7619
Epoch 3/200
15/15 [==============================] - 0s 6ms/step - loss: 0.4943 - accuracy: 0.8276 - val_loss: 0.5665 - val_accuracy: 0.8254
Epoch 4/200
15/15 [==============================] - 0s 5ms/step - loss: 0.4553 - accuracy: 0.8552 - val_loss: 0.5416 - val_accuracy: 0.8095
Epoch 5/200
15/15 [==============================] - 0s 6ms/step - loss: 0.4182 - accuracy: 0.8759 - val_loss: 0.5176 - val_accuracy: 0.8095
Epoch 6/200
15/15 [==============================] - 0s 6ms/step - loss: 0.3844 - accuracy: 0.8966 - val_loss: 0.4973 - val_accuracy: 0.8095
Epoch 7/200
15/15 [==============================] - 0s 6ms/step - loss: 0.3530 - accuracy: 0.9103 - val_loss: 0.4799 - val_accuracy: 0.8095
Epoch 8/200
